In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for 3.1.5_dps.py


In [2]:
from os import path
import os, glob
import datetime
import pandas as pd
import geopandas as gpd
!pip install xmltodict
import xmltodict
import collections
import sys
sys.path.append('/projects/Developer/icesat2_boreal/lib')
import ExtractUtils

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


### Build the input list of granules - s3fs implementation


In [13]:
if False:
    %%time
    import s3fs
    s3 = s3fs.S3FileSystem()
    bucket = "s3://maap-ops-workspace"
    searchkey_list = [f'nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/master/2022/03/**/*.tif' for DAY in ['16','17','18','19']]
    searchkey_list = [f'nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/master/2022/03/**/*.tif']
    # Concat list of lists to data frame
    topo_path_df = pd.concat([pd.DataFrame(s3.glob(os.path.join(bucket, searchkey)), columns=['maap_path']) for searchkey in searchkey_list])

    #Convert data frame to list
    INPUT_TILE_NUM_LIST = [i.replace("maap-ops-workspace", "https://maap-ops-workspace.s3.amazonaws.com") for i in topo_path_df.maap_path.to_list()]
    len(INPUT_TILE_NUM_LIST)

In [31]:
!python /projects/Developer/icesat2_boreal/lib/build_tindex_master.py -t Topo -y 2021 --dps_month_list 07 09 --maap_version ops
!python /projects/Developer/icesat2_boreal/lib/build_tindex_master.py -t Topo -y 2022 --dps_month_list 03 04 --maap_version master --tindex_append

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		ops
Type:		Topo
Year:		2021
Month:		['07', '09']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists
                                             s3_path  ...                                    file
0  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_1_covars_cog_topo_stack.tif
1  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_4_covars_cog_topo_stack.tif
2  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_5_covars_cog_topo_stack.tif
3  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_6_covars_cog_topo_stack.tif
4  s3://maap-ops-workspace/nathanmthomas/dps_outp...  

In [9]:
# Get all boreal tiles
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg' #shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg
boreal_tile_index = gpd.read_file(boreal_tile_index_path)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.

### Get all tiles

In [10]:
INPUT_TILE_NUM_LIST = boreal_tile_index.tile_num.to_list()
len(INPUT_TILE_NUM_LIST)

5337

### Get some subset of tiles

In [12]:
if False:
    need_tile_index_path = '/projects/my-public-bucket/DPS_tile_lists/Need_Topo_tindex_master.gpkg'
    need_tile_index = gpd.read_file(need_tile_index_path)

    if True:
        # Remove bad tiles
        need_tile_index = need_tile_index[~need_tile_index['tile_num'].isin(bad_tiles)]

    ax = boreal_tile_index.plot(color='gray')
    need_tile_index[need_tile_index.tile_version == 'version 2'].plot(column='tile_group', ax=ax)

    need_tile_index[need_tile_index.tile_version == 'version 2'].groupby('tile_version')['tile_group'].agg(['count'])

    INPUT_TILE_NUM_LIST = need_tile_index[need_tile_index.tile_version == 'version 2'].tile_num.to_list()

In [11]:
len(INPUT_TILE_NUM_LIST)

5337

In [27]:
TEST_DPS  = False

if TEST_DPS:
    print('Running DPS on a SUBSET list of input')
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST[0:100] #INPUT_TILE_NUM_LIST[100:]
else:
    print('Running DPS on the FULL list of input')
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
    
print(f"List length: {len(DPS_INPUT_TILE_NUM_LIST)}")

Running DPS on the FULL list of input
List length: 1


# Run a DPS of topo tiles

In [28]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'do_topo_stack_3-1-5'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'nathanmthomas'
    WORKER_TYPE = 'maap-dps-worker-8gb'
    
    in_param_dict = {
                         'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                         'in_tile_num': INPUT_TILE_NUM,
                         'tile_buffer_m': 120,
                         'in_tile_layer': 'boreal_tiles_v003',
                         'topo_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/dem30m_tiles.geojson',
        }

    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )
    
    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE
    
    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        
# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 1
DPS run #: 1	| tile num: 36469	| submit status: success	| job id: 6b894424-c44e-41ee-8b46-dade46fb731b
Current time:	202204211923
CPU times: user 17.7 ms, sys: 0 ns, total: 17.7 ms
Wall time: 277 ms


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,6b894424-c44e-41ee-8b46-dade46fb731b,1,36469,2022-04-21 19:23:37.368078,19,do_topo_stack_3-1-5_ubuntu,nathanmthomas,maap-dps-worker-8gb


After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [30]:
%%time
LIST_SUBMISSIONS = sorted(glob.glob(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_*.csv'),key=ExtractUtils.func, reverse=True)
for DPS_DATETIME in ['2022042119']:
    for fn in LIST_SUBMISSIONS:
        if DPS_DATETIME in fn and not 'job_status' in fn:
            DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
            thentime = fn.split('_')[-1].replace('.csv','')
            print(f'DPS alg:\t\t{DPS_alg_id}')
            print(f'DPS launch time:\t{thentime}')
            z = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
            # Save job status table
            z.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_job_status_{len(z)}_{thentime}.csv')

DPS alg:		do_topo_stack_3-1-5
DPS launch time:	202204211923
Count total jobs:	1
Count pending jobs:	0
Count running jobs:	0
Count succeeded jobs:	1
Count failed jobs:	0
% of failed jobs:	Nothing has failed...yet

DPS alg:		do_topo_stack_3-1-5
DPS launch time:	202204211923
Count total jobs:	7
Count pending jobs:	0
Count running jobs:	5
Count succeeded jobs:	1
Count failed jobs:	1
% of failed jobs:	50.0

DPS alg:		do_topo_stack_3-1-5
DPS launch time:	202204211907
Count total jobs:	7
Count pending jobs:	0
Count running jobs:	0
Count succeeded jobs:	6
Count failed jobs:	1
% of failed jobs:	14.29

CPU times: user 196 ms, sys: 14.1 ms, total: 211 ms
Wall time: 921 ms


In [23]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Succeeded'].iloc[0].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '9ab97df7-726f-44fd-a638-7c3c6b68104d'),
                           ('wps:Output',
                            OrderedDict([('@id',
                                          'output-2022-04-21T19:14:39.911298'),
                                         ('wps:Data',
                                          ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/master/2022/04/21/19/14/39/911298',
                                           's3

In [24]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[0].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            'e7db9b48-8251-49f3-a1c4-6ae7de444cd5'),
                           ('wps:Output',
                            OrderedDict([('@id', 'traceback'),
                                         ('wps:Data',
                                          'activate does not accept more than one argument:\n[\'/app/icesat2_boreal/dps/alg_3-1-5/run.sh\', \'36369\', \'120\', \'boreal_tiles_v003\']\n\n+ /app/icesat2_boreal/dps/alg_3-1-5/run.sh 36369 120 boreal_tiles_v003\n+ unset PROJ_LIB\n+ mkdir output\n+++ dirname /

### Update the tile index master csv

In [50]:
MAAP_VERSION = 'master'
OUTDIR = f'/projects/my-public-bucket/Topo/{MAAP_VERSION}'
!python /projects/Developer/icesat2_boreal/lib/build_tindex_master.py -t Topo -y 2022 -m 03 -o $OUTDIR --maap_version $MAAP_VERSION
MAAP_VERSION = 'ops'
OUTDIR = f'/projects/my-public-bucket/Topo/{MAAP_VERSION}'
#!python /projects/Developer/icesat2_boreal/lib/build_tindex_master.py -t Topo -y 2021 -m '*' -o $OUTDIR --maap_version $MAAP_VERSION

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		master
Type:		Topo
Year:		2022
Month:		03
Days:		1-31

Output dir:  /projects/my-public-bucket/Topo/master
                                             s3_path  ...                                        file
0  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_23548_covars_cog_topo_stack.tif
1  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_23547_covars_cog_topo_stack.tif
2  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_23549_covars_cog_topo_stack.tif
3  s3://maap-ops-workspace/nathanmthomas/dps_outp...  ...  Copernicus_23550_covars_cog_topo_stack.tif
4  s3://maap-ops-workspace/nathanmthom

# Combine Topo tindex master csvs from 'ops' and 'master', remove dups, and save to final list

In [51]:
out_tindex_fn = '/projects/my-public-bucket/DPS_tile_lists/Topo_tindex_master.csv'
fn_list = ['/projects/my-public-bucket/Topo/master/Topo_tindex_master.csv', '/projects/my-public-bucket/Topo/ops/Topo_tindex_master.csv']
df = pd.concat([pd.read_csv(fn) for fn in fn_list])
len(df)
duplicate_tiles = [item for item, count in collections.Counter(df["tile_num"]).items() if count > 1]
print(duplicate_tiles)
print(len(df))
df.to_csv(out_tindex_fn)
df.head()

[]
5072


,Unnamed: 0,s3_path,local_path,file,tile_num
0,26,s3://maap-ops-workspace/nathanmthomas/dps_outp...,/projects/my-private-bucket/dps_output/do_topo...,Copernicus_24109_covars_cog_topo_stack.tif,24109
1,89,s3://maap-ops-workspace/nathanmthomas/dps_outp...,/projects/my-private-bucket/dps_output/do_topo...,Copernicus_23551_covars_cog_topo_stack.tif,23551
2,90,s3://maap-ops-workspace/nathanmthomas/dps_outp...,/projects/my-private-bucket/dps_output/do_topo...,Copernicus_23550_covars_cog_topo_stack.tif,23550
3,91,s3://maap-ops-workspace/nathanmthomas/dps_outp...,/projects/my-private-bucket/dps_output/do_topo...,Copernicus_23833_covars_cog_topo_stack.tif,23833
4,92,s3://maap-ops-workspace/nathanmthomas/dps_outp...,/projects/my-private-bucket/dps_output/do_topo...,Copernicus_23219_covars_cog_topo_stack.tif,23219
